<a href="https://colab.research.google.com/github/mirab62807/DataScience/blob/main/Fianl_casecade_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook you'll create your own bootstrap function following the bootstrap algorithm (check the lecture notes!)

In [1]:
import matplotlib
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import math




In [19]:
# Load the data
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab2%20-%20bootstrap/customers.csv')
data = df.values.T[1]
print(data.shape[0])

14


In [17]:
# Checking the notes from the lecture, create here your own bootstrap function:
# 1. Sample from the input array x to create an array of samples of shape (n_bootstraps, sample_size)
# Hint: Check the function random.choice() on Numpy
# 2. Calculate and save the mean of the array (this is "data_mean" that is returned by the function)
# 3. Calculate the mean from each bootstrap (i.e., row) and store it.
# (This should be an array of n_bootstraps values)
# 4. Calculate the lower and upper bounds for a 95% CI (hint: check the percentile function on Numpy)
# 5. Return data_mean, and the lower and upper bounds of your interval
def bootstrap_mean(x, sample_size, n_bootstraps):
	bootstrap_matrix = []
	for i in range(n_bootstraps):
		boot_str = np.random.choice(data, sample_size)     #p=[1/sample_size]*sample_size)
		bootstrap_matrix.append(boot_str)
        
	bootstrap_array = np.array(bootstrap_matrix)
	#print(bootstrap_array)
	data_mean = np.mean(bootstrap_array, axis = 1)
	#print(data_mean)
	lower = np.percentile(data_mean, 2.5)
	upper = np.percentile(data_mean, 97.5)
	return data_mean, lower, upper




In [ ]:
# Call your bootstrap function and plot the results

boots = []
for i in range(100, 50000, 1000):
    boot = bootstrap_mean(data, data.shape[0], i)
    boots.append([i, boot[0], "mean"])
    boots.append([i, boot[1], "lower"])
    boots.append([i, boot[2], "upper"])


print(boots)

df_boot = pd.DataFrame(boots, columns=['Bootstrap Iterations', 'Mean', "Value"])
print(df_boot)
#print(df_boot.values.T[0])
#print(df_boot.values.T[1])
#sns_plot = sns.lmplot(df_boot.columns[0], df_boot.columns[1],data=df_boot )

#sns_plot.axes[0, 0].set_ylim(0,)
#sns_plot.axes[0, 0].set_xlim(0, 50000)



Now, modify the bootstrap function you created above so that you can pass your desired confidence interval as a parameter.



In [33]:
def bootstrap_mean_ci(data_from_file, data_size, n_bootstraps, ci):
  bootstrap_matrix = []
  take_arr = []
  for i in data_from_file:
    if math.isnan(i) == False:
      take_arr.append(i)

  size_no_nan = len(take_arr)
  no_nan_array = np.array(take_arr)
  for i in range (n_bootstraps):
    boot_str = np.random.choice(no_nan_array,size_no_nan)
    bootstrap_matrix.append(boot_str)

  bootstrap_array = np.array(bootstrap_matrix)
  
  data_mean = np.mean(bootstrap_array, axis = 1)
  data_mean_withpercentile = np.percentile(data_mean, ci)
  #print('with {} confidence interval mean is {}'.format(ci, data_mean_withpercentile))
  a = 100 - ci 
  b = a/2

  lower = np.percentile(data_mean, b)
  upper = np.percentile(data_mean,ci+b)
  return data_mean_withpercentile, lower, upper 



In [ ]:
boots = []
for i in range(100, 50000, 1000):
    boot = bootstrap_mean_ci(data, data.shape[0], i, 80)
    boots.append([i, boot[0], "mean"])
    boots.append([i, boot[1], "lower"])
    boots.append([i, boot[2], "upper"])

df_boot = pd.DataFrame(boots, columns=['Boostrap Iterations', 'Mean', "Value"])
sns_plot = sns.lmplot(df_boot.columns[0], df_boot.columns[1], data=df_boot, fit_reg=False, hue="Value")

sns_plot.axes[0, 0].set_ylim(0,)
sns_plot.axes[0, 0].set_xlim(0, 50000)

sns_plot.savefig("bootstrap_confidence_80.pdf", bbox_inches='tight')


# Vehicles dataset

Now let's work on a different dataset, which is stored in the vehicles.csv file.


In [ ]:
from IPython.core.pylabtools import figsize
# Load and visualise the vehicles dataset
# To load the dataset: https://neptune.ai/blog/google-colab-dealing-with-files (check section "Load individual files directly from GitHub")


# Note that the current and new fleets are in different columns and have different lengths, so bear this in mind when you're plotting.
# You can create separate scatterplots for the two fleets, as you would with the histograms, 
# or plot them both in one plot (but not one against the other).
# <---INSERT YOUR CODE HERE--->
# Note: you can add more cells as needed to organise your code and your plots

df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab2%20-%20bootstrap/vehicles.csv')
print(df.columns)
data1 = df['Current fleet'].values
#print(data1)
data2 = df['New Fleet'].values
#print(data2)
print(len(data2))
id = np.arange(1,len(df)+1)
df['id'] = id
print(len(df))
#print(df.head())
#sns.scatterplot(data=df, x="id", y="Current fleet")
#sns.scatterplot(data=df, x="id", y="New Fleet")

# shading because data1,data2 want to use but dont draw graph
'''
fig, ayes = plt.subplots(1,2, figsize=(15, 5), sharex=True)
fig.suptitle('Scatter Plot')
sns.scatterplot(ax=ayes[0], data=df, x="id", y="Current fleet")
sns.scatterplot(ax=ayes[1], data=df, x="id", y="New Fleet")

sns.displot(data=df.values.T[0], rug=False)
plt.xlabel("current fleet")
plt.ylabel("count")

sns.displot(data=df.values.T[1], rug=False)
plt.xlabel("New fleet")
plt.ylabel("count")'''

## Compare the two fleets

The business analysts come up a comparison algorithm that requires the upper and lower bounds for the mean in order to say which fleet is better.
1. Calculate the mean of both samples.
2. Using the bootstrap function that you created:
    - Construct the 95% CI of the mean of the current fleet.
    - Construct the 95% CI of the mean of the new fleet.
    - Are they comparable? (i.e., is one better than the other?) -- you can do this with a permutation test (check the lecture notes!)

In [53]:
# <---INSERT YOUR CODE HERE--->
# A function that return only mean of current and new fleet
def mean_of_fleets(data_of_fleets, data_size, n_bootstraps):
  take_arr = []
  for i in data2:
    if math.isnan(i) == False:
      take_arr.append(i)

  size_no_nan = len(take_arr)
  no_nan_array = np.array(take_arr)
  store_matrix = []
  for i in range (n_bootstraps):
    boot_str = np.random.choice(no_nan_array,size_no_nan)
    store_matrix.append(boot_str)
  store_array = np.array(store_matrix)
  data_mean = np.mean(store_array, axis = 1)
  return data_mean 

def mean_and_bootstrapmean_ci(data_current_fleet,data_new_fleet, data_size, n_bootstraps, ci):
  mean_of_current_fleet = mean_of_fleets(data_current_fleet, data_size, n_bootstraps)
  mean_of_new_fleet = mean_of_fleets(data_new_fleet, data_size, n_bootstraps)
  print('mean of current fleet= {}'.format(mean_of_current_fleet))
  print('mean of new fleet = {}'.format(mean_of_new_fleet))

  with_ci_mean_current_fleet = bootstrap_mean_ci(data_current_fleet, data_size, n_bootstraps, ci)
  with_ci_mean_new_fleet = bootstrap_mean_ci(data_new_fleet, data_size, n_bootstraps, ci)
  print('with {} mean of current fleet = {}'.format(ci,with_ci_mean_current_fleet))
  print('with {} mean of new fleet = {}'.format(ci,with_ci_mean_new_fleet))


mean_and_bootstrapmean_ci(data1, data2, len(df),10, 95 )


mean of current fleet= [31.30379747 30.58227848 30.06329114 31.03797468 29.79746835 31.24050633
 29.92405063 30.01265823 30.74683544 31.4556962 ]
mean of new fleet = [31.39240506 29.92405063 29.29113924 31.62025316 29.40506329 30.72151899
 30.73417722 29.73417722 30.46835443 31.18987342]
with 95 mean of current fleet = (20.467670682730926, 19.46937751004016, 20.48483935742972)
with 95 mean of new fleet = (31.26392405063291, 29.651582278481015, 31.34082278481013)


In [68]:
# Create your own function for a permutation test here (you will need it for the lab quiz!):
def remove_nan(sample):
  take_arr = []
  for i in sample:
    if math.isnan(i) == False:
        take_arr.append(i)
  size_no_nan = len(take_arr)
  no_nan_array = np.array(take_arr)
  return no_nan_array


def permut_test(sample1, sample2, n_permutations):
    """
    sample1: 1D array
    sample2: 1D array (note that the size of the two arrays can be different)
    n_permutations: number of permutations to calculate the p-value
    """
    data_current = remove_nan(sample1)
    data_new = remove_nan(sample2)
    mean_of_current = mean_of_fleets(data_current,len(data_current), n_permutations)
    mean_of_new = mean_of_fleets(data_new, len(data_new),n_permutations)
    Tobs = mean_of_new - mean_of_current
    print(Tobs)


    
    concat = np.concatenate((data_current, data_new))
    for i in range(n_permutations):
      perm = np.random.permutation(concat)
      #print(concat)
      #print(perm)
      p_current = perm [:int(len(data_current))]
      p_new = perm[int(len(data_new)):]
      #print(len(p_current),len(p_new))
      mean_curr_af_perm = mean_of_fleets(p_current,len(p_current), 10)
      mean_new_af_perm = mean_of_fleets(p_new, len(p_new), 10)
      Tperm = mean_new_af_perm - mean_curr_af_perm
      print('for {} the Tperm is{}='.format(i,Tperm))
    count = 0
    for i in range(n_permutations):
      if Tperm[i]>Tobs[i]:
        count = count +1
    pvalue = n_permutations/count     
    return pvalue


print(permut_test(data1, data2, 10))



[-0.20253165  0.35443038  0.74683544 -0.43037975 -0.27848101  0.3164557
 -1.98734177  1.20253165 -0.93670886  1.02531646]
for 0 the Tperm is[ 0.50632911  1.07594937  0.07594937  1.06329114  0.58227848 -1.37974684
  0.37974684  0.55696203  1.70886076 -0.24050633]=
for 1 the Tperm is[-1.06329114  0.16455696  1.37974684 -0.6835443  -1.2278481   2.79746835
  0.69620253 -1.25316456  0.4556962   0.34177215]=
for 2 the Tperm is[-1.5443038  -0.13924051 -0.62025316  1.06329114 -0.86075949 -0.30379747
 -1.11392405  1.17721519 -1.10126582  0.86075949]=
for 3 the Tperm is[-0.62025316 -0.15189873  1.02531646  0.35443038 -1.5443038   0.32911392
  0.06329114  1.15189873  0.         -1.82278481]=
for 4 the Tperm is[ 1.78481013 -1.39240506  0.37974684 -0.06329114  0.56962025 -0.50632911
  2.39240506  0.26582278  1.36708861 -0.73417722]=
for 5 the Tperm is[ 0.75949367  1.12658228 -0.5443038   1.67088608  0.39240506 -2.21518987
 -0.74683544 -0.4556962  -0.72151899  1.06329114]=
for 6 the Tperm is[ 1.5696